# Kafka and Spark structured streaming

## Kafka server setup
Folow the instructions on https://www.baeldung.com/ops/kafka-docker-setup to create a local single node kafka cluster.

## Kafka producer
Clone the SimpleKafkaProducer available on https://gitlab.com/kdg-ti/dataengineering/kafkaproducer
This producer simulates a very simple wordcount with 3 words. The timestamps are randomly lagging. This makes it possible to look at how late arriving data is used.


In [3]:
import pyspark
from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip
import ConnectionConfig as cc

## Spark session setup
Spark needs extra jars to be able to run the jobs. Change the pathlocations for your installation.

In [4]:



builder = SparkSession.builder \
    .appName("Kafka") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.sql.shuffle.partitions", "4") \
    .config("spark.jars", cc.jars_loc + "kafka-clients-2.6.0.jar," \
                          + cc.jars_loc + "commons-pool2-2.6.2.jar," \
                          + cc.jars_loc + "mssql-jdbc-10.2.1.jre8.jar," \
                          + cc.jars_loc + "spark-sql-kafka-0-10_2.12-3.1.3.jar," \
                          + cc.jars_loc + "spark-tags_2.12-3.1.3.jar, " \
                          + cc.jars_loc + "spark-token-provider-kafka-0-10_2.12-3.1.3.jar") \
    .master("local[*]")

builder = configure_spark_with_delta_pip(builder)
spark = builder.getOrCreate()
builder.getOrCreate()

In [5]:
print(spark._jsc.sc().listJars())

Vector(spark://AKDGPORT11191.mshome.net:64087/jars/kafka-clients-2.6.0.jar, spark://AKDGPORT11191.mshome.net:64087/jars/commons-pool2-2.6.2.jar, spark://AKDGPORT11191.mshome.net:64087/jars/spark-sql-kafka-0-10_2.12-3.1.3.jar, spark://AKDGPORT11191.mshome.net:64087/jars/mssql-jdbc-10.2.1.jre8.jar, spark://AKDGPORT11191.mshome.net:64087/jars/spark-tags_2.12-3.1.3.jar, spark://AKDGPORT11191.mshome.net:64087/jars/spark-token-provider-kafka-0-10_2.12-3.1.3.jar)


## Reading from a streaming data source
We start a readStream on a kafka data source. The server is the one you are running on docker.
The subscribe option is needed to define the Topic we are going to read.

The incoming messages contain a key and a value in binary format. Other metadata is included. In this specific case the meta data itself is not used

### Transforming the "value" column
The value column is in binary format. It has to be cast to String first. The producer creates json records. json_tuple is used to convert the json elements to seperate columns in string format.
The result is converted to a timestamp, key and value column.

### Watermarking
A watermark is added based on the timestamp field (this field is the effective date on wich the event occured). The delayThreshold is used to set the allowed delay of arrival of

### Aggregating the data
the data is grouped by word and time windows. Timewindows can be definded with the window function.

### Writing to a sink
The "console" format is used for debugging. Output mode is "update" because it will take until after the watermarking threshold to persist any information. Setting the checkpoint location is needed because spark has to create checkpoints in order to start the job again.
Tip: If the job starts giving errors on the checkpoint, you can delete the directory and start over again.

In [7]:
from pyspark.sql import functions as F
df = spark \
      .readStream \
      .format("kafka") \
      .option("kafka.bootstrap.servers", "AKDGPORT11191.mshome.net:29092" ) \
      .option("subscribe", "demo") \
      .load()

In [6]:
df.printSchema()
#df.writeStream.format("console").outputMode("update").option("checkpointLocation",".\checkpoints").start("kafkaTest")
inter = df.selectExpr("json_tuple(CAST(value as STRING), 'timestamp', 'key', 'value')")
inter.printSchema()
wordCountEvents= inter.selectExpr("to_timestamp(trim('[]' FROM c0),'yyyy,M,d,H,m,s,SSSSSSSSS') as timestamp", "c1 as key", "c2 as value").withWatermark("timestamp",'30 seconds')
wordCountEvents.printSchema()
groupedEvents =wordCountEvents.groupBy(F.window(wordCountEvents.timestamp, "2 minutes", "2 minutes"), wordCountEvents.key) \
 .count()

groupedEvents.writeStream.format("console").outputMode("update").option("checkpointLocation",".\checkpoints").start("kafkaTest")

#{"timestamp":[2022,10,5,17,26,17,470412000],"key":"Fish","value":1}

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)

root
 |-- c0: string (nullable = true)
 |-- c1: string (nullable = true)
 |-- c2: string (nullable = true)

root
 |-- timestamp: timestamp (nullable = true)
 |-- key: string (nullable = true)
 |-- value: string (nullable = true)



## Stop the stream
If you do not stop the stream in code, the job wil keep on running.

In [5]:
spark.stop()